In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean as _mean, stddev as _stddev, col , collect_list
from IPython.display import display


In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Program Files\\Spark\\spark-3.5.1-bin-hadoop3'

In [3]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("My Spark App") \
    .getOrCreate()

# load data

In [4]:
cleaned_df = spark.read.csv("cleaned_songs.csv", header=True, inferSchema=True)

print(cleaned_df.count())

17536


# prepare data for classification

In [5]:
# remove lyrics

cleaned_df = cleaned_df.drop("lyrics")


# normalize features (danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo) using z-scoring
features = [
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "duration_ms",
]

for feature in features:
    stats = cleaned_df.select(
        _mean(col(feature)).alias("mean"), _stddev(col(feature)).alias("stddev")
    ).collect()
    mean = stats[0]["mean"]
    stddev = stats[0]["stddev"]

    # Normalize each feature using z-scoring
    cleaned_df = cleaned_df.withColumn(feature, (col(feature) - mean) / stddev)

    # make all values lies between -1 and 1

    min_value = cleaned_df.agg({feature: "min"}).collect()[0][0]
    max_value = cleaned_df.agg({feature: "max"}).collect()[0][0]

    value = max(abs(min_value), abs(max_value))

    cleaned_df = cleaned_df.withColumn(feature, (col(feature) / value))

# export to csv

cleaned_df.toPandas().to_csv("normalized_songs.csv", index=False)

In [6]:
from sklearn.model_selection import train_test_split

# transform the dataframe into an array of features

data = (
    cleaned_df.select(
        "playlist_genre",
        "danceability",
        "energy",
        # "key",
        "loudness",
        # "mode",
        "speechiness",
        "acousticness",
        # "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        "duration_ms",
    )
    .toPandas()
    .to_numpy()
)

# split the data into train and test sets

train_data, test_Data, train_labels, test_labels = train_test_split(
    data[:, 1:], data[:, 0], test_size=0.2, random_state=42
)

# SVM

### best score: 0.5236602052451539

In [7]:
from sklearn.svm import SVC
# train the model
model = SVC(kernel='rbf' )

model.fit(train_data, train_labels)


SVC()

In [8]:

# evaluate the model

score = model.score(test_Data, test_labels)

print (score)
    

0.5433295324971493


# Random Forests

### best score: 0.5653010723123454

In [9]:
from sklearn.ensemble import RandomForestClassifier

model =RandomForestClassifier(
    n_estimators=300,
    criterion="entropy",
    max_depth=20,
    min_samples_split=2,
)

# train the model
model.fit(train_data, train_labels)

RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=300)

In [10]:
# evaluate the model

score = model.score(test_Data, test_labels)
print (score)

0.5667046750285063


# decision tree

### best score: 0.47594171020071485

In [16]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(
    criterion='gini', 
    splitter='best', 
    max_depth=10, 
    min_samples_split=50, 
    min_samples_leaf=10, 
    min_weight_fraction_leaf=0.0, 
    max_features=None, 
    random_state=None, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    class_weight=None, 
    ccp_alpha=0.0
)
clf.fit(train_data, train_labels)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=50)

In [17]:
score = clf.score(test_Data, test_labels)

print (score)

0.47633979475484606
